In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd


file = pd.ExcelFile('Wijkdata_Jeugdhulp_in_de_wijk.ods')

sheet_names = file.sheet_names

wijk = pd.read_excel(file, sheet_name=sheet_names[-3])  # wijk
gebr = pd.read_excel(file, sheet_name=sheet_names[-2])  # gebruik
det = pd.read_excel(file, sheet_name=sheet_names[-1])  # determinanten

file.close()

df = det.copy()
# als je wijk en gemeentecode hebt heb je wijkcode en gemeentenaam niet nodig
df = df.merge(wijk[['wijk', 'gemeentecode']], on='wijk', how='left')
# per_jhzv : aandeel jeugdigen met jeugdhulp zonder verblijf, waargenomen -> target variable
df = df.merge(gebr[['wijk', 'perc_jhzv']], on='wijk', how='left')

for col in df.columns:
    df[col] = df[col].apply(lambda x: x if x != '.' else np.nan)

# TODO is nu missig values vullen met gemiddelde hoe willen we dat doen?
df.fillna(df.mean(), inplace=True)

# hun schatting root mean squared error (residu = schatting - waargenomen)
for col in gebr.columns:
    gebr[col] = gebr[col].apply(lambda x: x if x != '.' else np.nan)

# drop missende schattingen en bereken RMSE
original_rmse = str((gebr.dropna(subset='residu')['residu']**2).mean()**0.5)

# buurt_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/buurt_2022_v1.shp')
wijk_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/wijk_2022_v1.shp')
# gemeente_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/gemeente_2022_v1.shp')

wijk_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/wijk_2022_v1.shp')
wijk_geo['WK_CODE'] = wijk_geo['WK_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_geo['GM_CODE'] = wijk_geo['GM_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_final = df.merge(wijk_geo, left_on=['wijk', 'gemeentecode'], right_on=['WK_CODE', 'GM_CODE'], how='left')
for col in wijk_final.columns:
    wijk_final[col] = wijk_final[col].apply(lambda x: x if x != int(-99999999) else np.nan)

df = wijk_final.fillna(wijk_final.mean())
df = df.drop(['JRSTATCODE', 'JAAR', 'Shape_Leng', 'Shape_Area', 'geometry', 'wijk', 'WK_CODE', 'WK_NAAM', 'GM_CODE', 'GM_NAAM', 'gemeentecode', 'H2O'], axis=1)
df

FileNotFoundError: [Errno 2] No such file or directory: 'Wijkdata_Jeugdhulp_in_de_wijk.ods'

In [117]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler


X = df.drop('perc_jhzv', axis=1)

y = df['perc_jhzv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = RobustScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Transform both the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Instantiate the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(X_test)

# Calculate RMSE of the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Model RMSE : ", rmse)
print("their Model RMSE: ", original_rmse)

Model RMSE :  2.8433170791904403
their Model RMSE:  2.2551381084753706


In [125]:
from sklearn.linear_model import LassoCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = RobustScaler()

lasso_cv = LassoCV(alphas=None, cv=10, max_iter=10000)

lasso_cv.fit(X_train, y_train)
y_pred = lasso_cv.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

3.183056246761253

In [126]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the training data
rf.fit(X_train, y_train)

# Predict the prices for the test data
predictions = rf.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Mean Squared Error: {rmse}')

In [99]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Create DMatrix for train and test
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameters
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'reg:squarederror'}  # regression with squared loss

num_round = 50  # the number of training iterations

# Train XGBoost model
bst = xgb.train(param, dtrain, num_round)

# Predictions
preds = bst.predict(dtest)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f'Mean Squared Error: {rmse}')


Mean Squared Error: 2.8058677510709105


In [91]:
df.columns

Index(['perc_sbo_g', 'perc_vso_g', 'perc_praktijk_g', 'perc_hhmk',
       'perc_jong04', 'perc_1ouder', 'perc_tienmam', 'perc_hhmkhgink',
       'perc_kind_sch5', 'perc_ouderpsy', 'perc_hhverd', 'perc_hhbijs',
       'perc_hhwan', 'groenedruk', 'grijzedruk', 'perc_ao', 'perc_laagopl',
       'gem_gestink', 'polchrplus', 'perc_nwmig', 'perc_antsur', 'perc_1gez',
       'huur_corp', 'woonwaarde', 'afstandhuisarts', 'samenw', 'kleinewijk',
       'perc_jhzv', 'IND_WBI', 'OAD', 'STED', 'BEV_DICHTH', 'AANT_INW',
       'AANT_MAN', 'AANT_VROUW', 'P_00_14_JR', 'P_15_24_JR', 'P_25_44_JR',
       'P_45_64_JR', 'P_65_EO_JR', 'P_ONGEHUWD', 'P_GEHUWD', 'P_GESCHEID',
       'P_VERWEDUW', 'AANTAL_HH', 'P_EENP_HH', 'P_HH_Z_K', 'P_HH_M_K',
       'GEM_HH_GR', 'P_WEST_AL', 'P_N_W_AL', 'P_MAROKKO', 'P_ANT_ARU',
       'P_SURINAM', 'P_TURKIJE', 'P_OVER_NW', 'OPP_TOT', 'OPP_LAND',
       'OPP_WATER'],
      dtype='object')